In [ ]:
#175. Combine Two Tables
SELECT firstName , lastName , city , state
FROM person P
LEFT JOIN address A 
ON P.personId = A.personId

In [ ]:
#176. Second Highest Salary
WITH
tbl AS (
    SELECT DISTINCT salary SecondHighestSalary FROM (
        SELECT * , DENSE_RANK() OVER (ORDER BY salary DESC) DR FROM Employee) RNK
    WHERE DR = 2)

SELECT * FROM TBL
UNION
SELECT NULL
WHERE NOT EXISTS (SELECT * FROM tbl)

In [ ]:
#177. Nth Highest Salary
CREATE FUNCTION getNthHighestSalary(@N INT) RETURNS INT AS
BEGIN
    RETURN (
        SELECT DISTINCT salary
        FROM(
            SELECT salary, DENSE_RANK() OVER (ORDER BY salary DESC) DR
            FROM Employee) A
        WHERE DR = @N
    );
END

In [ ]:
#178. Rank Scores
SELECT score, DENSE_RANK() OVER (ORDER BY score DESC) rank
FROM Scores

In [ ]:
#180. Consecutive Numbers
SELECT DISTINCT num ConsecutiveNums 
FROM(
    SELECT num, LEAD(NUM, 1) OVER (ORDER BY id) L1,
    LEAD(NUM, 2) OVER (ORDER BY id) L2
    FROM Logs) A
WHERE num = L1 AND L1 = L2

In [ ]:
#181. Employees Earning More Than Their Managers
SELECT E1.name Employee FROM employee E1
LEFT JOIN employee E2 ON E1.managerId = E2.id
WHERE E1.salary > E2.salary

In [ ]:
#182. Duplicate Emails
SELECT email Email
FROM Person
GROUP BY email
HAVING COUNT(email) > 1

In [ ]:
#183. Customers Who Never Order
SELECT name AS Customers FROM customers
WHERE id NOT IN (SELECT customerId FROM Orders)

In [ ]:
#184. Department Highest Salary
SELECT D.name Department, E.name Employee, E.salary Salary
FROM (
  SELECT name, salary, departmentId, MAX(salary) OVER (PARTITION BY departmentId) max_sal
  FROM Employee) E
LEFT JOIN Department D ON E.departmentId=D.id
WHERE E.max_sal = E.salary

In [ ]:
#185. Department Top Three Salaries
SELECT D.name Department, E.Employee, E.Salary
FROM (
    SELECT departmentId Department, name Employee, salary Salary, 
    DENSE_RANK() OVER (PARTITION BY departmentId ORDER BY salary DESC) DR
    FROM Employee) E
LEFT JOIN Department D ON E.Department = D.id
WHERE DR <= 3

In [ ]:
#196. Delete Duplicate Emails
DELETE FROM Person
WHERE id NOT IN (SELECT MIN(ID) FROM person GROUP BY email)

In [ ]:
#197. Rising Temperature
SELECT id AS Id
FROM (
    SELECT * , 
    LAG(temperature) OVER (ORDER BY recordDate) LAGTEMP ,
    LAG(recordDate) OVER (ORDER BY recordDate) LAGdate    
    FROM Weather) A
WHERE temperature > LAGTEMP AND DATEADD(DD,-1,recordDate) = LAGdate

In [ ]:
#262. Trips and Users
WITH
banned AS (
    SELECT users_id 
    FROM Users
    WHERE banned = 'Yes'),

total AS (
    SELECT request_at, COUNT(id) cnt_tot
    FROM Trips
    WHERE client_id NOT IN (SELECT * FROM banned) 
    AND driver_id NOT IN (SELECT * FROM banned)
    GROUP BY request_at),

cancelled AS (
    SELECT request_at, COUNT(id) cnt_cnl
    FROM Trips
    WHERE status != 'completed' 
    AND client_id NOT IN (SELECT * FROM banned) 
    AND driver_id NOT IN (SELECT * FROM banned)
    GROUP BY request_at)

SELECT T.request_at Day, 
ROUND(CAST(COALESCE(C.cnt_cnl, 0) AS FLOAT)/T.cnt_tot, 2) [Cancellation Rate]
FROM total T
FULL JOIN cancelled C ON T.request_at=C.request_at
WHERE T.cnt_tot != 0 
AND (T.request_at <= CAST('2013-10-03' AS DATE) AND T.request_at >= CAST('2013-10-01' AS DATE))

In [ ]:
#511. Game Play Analysis I
SELECT player_id, MIN(event_date) first_login FROM Activity
GROUP BY player_id

In [ ]:
#584. Find Customer Referee
SELECT name FROM customer
WHERE referee_id!=2 OR referee_id IS NULL

In [ ]:
#586. Customer Placing the Largest Number of Orders
WITH
CNT_O AS (
    SELECT customer_number , COUNT(order_number) CNT_O
    FROM Orders A
    GROUP BY customer_number)

SELECT customer_number FROM (
    SELECT customer_number , CNT_O , DENSE_RANK() OVER (ORDER BY CNT_O DESC) DR
    FROM CNT_O) A
WHERE DR = 1

In [ ]:
#595. Big Countries
SELECT name , population , area 
FROM world
WHERE area > 3000000 OR population >= 25000000

In [ ]:
#596. Classes More Than 5 Students
SELECT class
FROM Courses
GROUP BY class
HAVING COUNT(student) >= 5

In [ ]:
#601. Human Traffic of Stadium
SELECT id, visit_date, people
FROM (
    SELECT id, visit_date, people, 
    CASE WHEN 
    (LAG(id) OVER (ORDER BY id) IS NULL AND LEAD(id) OVER (ORDER BY id) = id + 1 AND LEAD(id,2) OVER (ORDER BY id) = id + 2)
    OR
    (LEAD(id) OVER (ORDER BY id) IS NULL AND LAG(id) OVER (ORDER BY id) = id - 1 AND LAG(id,2) OVER (ORDER BY id) = id - 2)
    OR
    (LAG(id, 1) OVER (ORDER BY id) = id - 1 AND LEAD(id) OVER (ORDER BY id) = id + 1)
    OR
    (LEAD(id, 1) OVER (ORDER BY id) = id + 1 AND LEAD(id, 2) OVER (ORDER BY id) = id + 2)
    OR
    (LAG(id, 1) OVER (ORDER BY id) = id - 1 AND LAG(id, 2) OVER (ORDER BY id) = id - 2)
    THEN 1
    ELSE 0
    END ind
    FROM (
        SELECT *
        FROM Stadium
        WHERE people >= 100
    ) A
) B
WHERE ind = 1

In [ ]:
#607. Sales Person
WITH
A AS (
    SELECT SP.name SPN , C.name CN
    FROM SalesPerson SP
    LEFT JOIN Orders O ON SP.sales_id=O.sales_id
    LEFT JOIN company C ON O.com_id=C.com_id)

SELECT name FROM SalesPerson
WHERE name NOT IN (SELECT SPN FROM A WHERE CN='RED')

In [ ]:
#608. Tree Node
SELECT id ,
CASE
WHEN p_id IS NULL
THEN 'Root'
WHEN id IN (SELECT p_id FROM tree) AND p_id IS NOT NULL
THEN 'Inner'
ELSE 'Leaf'
END type
FROM Tree

In [ ]:
#620. Not Boring Movies
SELECT * FROM Cinema
WHERE id % 2 != 0 AND description != 'boring'
ORDER BY rating DESC

In [ ]:
#626. Exchange Seats
SELECT 
COALESCE(
    CASE 
    WHEN id % 2 != 0
    THEN LEAD(id) OVER (ORDER BY id)
    ELSE LAG(id) OVER (ORDER BY id)
    END, id) id,
student
FROM Seat
ORDER BY id

In [ ]:
#627. Swap Salary
UPDATE Salary
SET SEX =
CASE
WHEN SEX = 'm'
THEN 'f'
ELSE 'm'
END 

In [ ]:
#1050. Actors and Directors Who Cooperated At Least Three Times
SELECT actor_id, director_id
FROM ActorDirector
GROUP BY actor_id, director_id
HAVING COUNT(*) >= 3

In [ ]:
#1084. Sales Analysis III
SELECT DISTINCT S.product_id, P.product_name 
FROM Sales S
LEFT JOIN Product P ON S.product_id=P.product_id
WHERE 
sale_date >= CAST('2019-01-01' AS DATE) 
AND sale_date <= CAST('2019-03-31' AS DATE)
AND S.product_id NOT IN (
    SELECT product_id FROM Sales
    WHERE sale_date < CAST('2019-01-01' AS DATE) OR sale_date > CAST('2019-03-31' AS DATE))

In [ ]:
#1141. User Activity for the Past 30 Days I
SELECT activity_date day, COUNT(DISTINCT user_id) active_users 
FROM (
    SELECT user_id , activity_date FROM Activity
    WHERE 
    DATEDIFF(DD,activity_date,CAST('2019-07-27' AS DATE)) < 30 
    AND DATEDIFF(DD,activity_date,CAST('2019-07-27' AS DATE)) > 0
    GROUP BY user_id , activity_date) A
GROUP BY activity_date

In [ ]:
#1148. Article Views I
SELECT DISTINCT author_id id FROM Views
WHERE author_id = viewer_id

In [ ]:
#1158. Market Analysis I
WITH
Buyer AS (
    SELECT buyer_id, COUNT(*) orders_in_2019
    FROM Orders
    WHERE DATEPART(YEAR, order_date) = 2019
    GROUP BY buyer_id)

SELECT user_id buyer_id, join_date, COALESCE(orders_in_2019, 0) orders_in_2019
FROM Users U
LEFT JOIN Buyer B ON U.user_id = B.buyer_id

In [ ]:
#1179. Reformat Department Table
SELECT id,
SUM(CASE month WHEN 'Jan' THEN revenue ELSE NULL END) Jan_Revenue,
SUM(CASE month WHEN 'Feb' THEN revenue ELSE NULL END) Feb_Revenue,
SUM(CASE month WHEN 'Mar' THEN revenue ELSE NULL END) Mar_Revenue,
SUM(CASE month WHEN 'Apr' THEN revenue ELSE NULL END) Apr_Revenue,
SUM(CASE month WHEN 'May' THEN revenue ELSE NULL END) May_Revenue,
SUM(CASE month WHEN 'Jun' THEN revenue ELSE NULL END) Jun_Revenue,
SUM(CASE month WHEN 'Jul' THEN revenue ELSE NULL END) Jul_Revenue,
SUM(CASE month WHEN 'Aug' THEN revenue ELSE NULL END) Aug_Revenue, 
SUM(CASE month WHEN 'Sep' THEN revenue ELSE NULL END) Sep_Revenue, 
SUM(CASE month WHEN 'Oct' THEN revenue ELSE NULL END) Oct_Revenue, 
SUM(CASE month WHEN 'Nov' THEN revenue ELSE NULL END) Nov_Revenue, 
SUM(CASE month WHEN 'Dec' THEN revenue ELSE NULL END) Dec_Revenue
FROM Department
GROUP BY id

In [ ]:
#1393. Capital Gain/Loss
SELECT stock_name,
SUM(IIF(operation = 'Sell', price, price*(-1))) capital_gain_loss
FROM Stocks
GROUP BY stock_name
ORDER BY capital_gain_loss DESC

In [ ]:
#1407. Top Travellers
SELECT U.name, COALESCE(SUM(R.distance), 0) travelled_distance
FROM Rides R
RIGHT JOIN Users U ON R.user_id=U.id
GROUP BY R.user_id, U.name
ORDER BY SUM(R.distance) DESC, U.name

In [ ]:
#1484. Group Sold Products By The Date
SELECT sell_date , COUNT(product) num_sold , STRING_AGG(product,',') WITHIN GROUP (ORDER BY product) products
FROM (SELECT DISTINCT * FROM Activities) A
GROUP BY sell_date
ORDER BY sell_date

In [ ]:
#1527. Patients With a Condition
SELECT * FROM Patients
WHERE conditions LIKE'% DIAB1%' OR conditions LIKE'DIAB1%'

In [ ]:
#1581. Customer Who Visited but Did Not Make Any Transactions
SELECT customer_id , COUNT(customer_id) count_no_trans
FROM Visits V
LEFT JOIN Transactions T 
ON V.visit_id = T.visit_id
WHERE transaction_id IS NULL
GROUP BY customer_id

In [ ]:
#1587. Bank Account Summary II
SELECT U.name, SUM(T.amount) balance
FROM Transactions T
LEFT JOIN Users U ON T.account = U.account
GROUP BY T.account, U.name
HAVING SUM(T.amount) > 10000

In [ ]:
#1667. Fix Names in a Table
SELECT user_id , SUBSTRING(UPPER(name),1,1) + SUBSTRING(LOWER(name),2,254) name
FROM users
ORDER BY user_id

In [ ]:
#1693. Daily Leads and Partners
SELECT date_id , make_name , COUNT(DISTINCT lead_id) unique_leads , COUNT(DISTINCT partner_id) unique_partners 
FROM DailySales
GROUP BY date_id , make_name

In [ ]:
#1729. Find Followers Count
SELECT user_id , COUNT(follower_id) followers_count
FROM Followers
GROUP BY user_id
ORDER BY user_id

In [ ]:
#1741. Find Total Time Spent by Each Employee
SELECT event_day day, emp_id, SUM(out_time - in_time) total_time
FROM Employees
GROUP BY event_day, emp_id

In [ ]:
#1757. Recyclable and Low Fat Products
SELECT product_id FROM products
WHERE low_fats  = 'Y' AND recyclable = 'Y'

In [ ]:
#1795. Rearrange Products Table
SELECT product_id , 'store1' AS store , store1 AS price
FROM products
WHERE store1 IS NOT NULL

UNION

SELECT product_id , 'store2' AS store , store2
FROM products
WHERE store2 IS NOT NULL

UNION

SELECT product_id , 'store3' AS store , store3
FROM products
WHERE store3 IS NOT NULL

In [ ]:
#1873. Calculate Special Bonus
SELECT employee_id , IIF(employee_id % 2 != 0 AND name NOT LIKE 'M%', salary , 0) bonus 
FROM Employees
ORDER BY employee_id

In [ ]:
#1890. The Latest Login in 2020
SELECT DISTINCT user_id, MAX(time_stamp) OVER (PARTITION BY user_id) last_stamp
FROM Logins
WHERE DATEPART(YYYY,time_stamp) = 2020

In [ ]:
#1965. Employees With Missing Information
SELECT COALESCE(E.employee_id,S.employee_id) employee_id
FROM Employees E
FULL JOIN Salaries S ON E.employee_id = S.employee_id
WHERE name IS NULL OR salary IS NULL
ORDER BY employee_id